# GBC Replication

The following notebook will attemp to use GBC and partial dependance plots to explain the features used in the paper:https://arxiv.org/pdf/1610.05359v2.pdf In which XGBoost Classifier was trained to recognize whether tightly packed planetary systems were stable or not. 

To do this the Gradient Boosting Classifier will be tuned to mimic the scores the author obtained and then utilize  the Partial Dependance Plot functions which can demonstrate the interactions between features of a Boosted Tree based classifier; this is nessissary as XGBoost is not compatible with the current sklearn partial dependance plot  implementation. 

In [30]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn import cross_validation, metrics   #Additional scklearn functions
import matplotlib.pyplot as plt
%matplotlib inline
trainData = pd.read_csv('1e7data_train.csv', index_col=0)
testData = pd.read_csv('1e7data_holdout.csv', index_col=0)
df = trainData
dftest = testData

In [31]:
features = []
features += ['daOverRH1', 'daOverRH2']
features += ['mindaOverRH', 'maxdaOverRH']
features += ['norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1']
features += ['norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2']
features += ['norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3']
features += ['avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1']
features += ['avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2']
features += ['avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3']
features += ['norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope']
features += ['norm_LyapunovTime']


X_train = df[features]

y_train = df['Stable']
X_test = dftest[features]

X_train = X_train.replace(np.nan, -999)
X_train = X_train.replace(np.inf, -999)

X_test = X_test.replace(np.nan, -999)
X_test = X_test.replace(np.inf, -999)

In [32]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

model = GradientBoostingClassifier(loss='deviance', learning_rate=0.002, n_estimators=5000,
                                  subsample=.5,  min_samples_split=2, 
                                  min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=8,
                                  init=None, random_state=42, 
                                  max_leaf_nodes=8, warm_start=False, max_features=.5)

model.fit(X_train, y_train)

dtest_predictions = model.predict(X_test) 
dtest_predprob = model.predict_proba(X_test)[:, 1]

#Print model report:
print "\nModel Report"
print "Accuracy : %.4g" % metrics.accuracy_score(dftest['Stable'].values, dtest_predictions)
print "AUC Score (Test): %f" % metrics.average_precision_score(dftest['Stable'], dtest_predprob)
print 'ROC_AUC_Score: ', metrics.roc_auc_score(dftest['Stable'], dtest_predprob)




Model Report
Accuracy : 0.882
AUC Score (Test): 0.895542
ROC_AUC_Score:  0.945628156566


In [33]:
import xgboost
from xgboost.sklearn import XGBClassifier 

model0 = XGBClassifier(colsample_bylevel=1, colsample_bytree=1,
                         n_estimators=5000, gamma=0,
                         max_delta_step=0, missing=None, 
                         reg_alpha=0, reg_lambda=1,
                         scale_pos_weight=1, seed=27,
                        learning_rate=.002, max_depth=8,
                        subsample=.5, min_child_weight=1.2, 
                        objective='binary:logistic', nthread=-1)

model0.fit(X_train, y_train)

dtest_predictions = model0.predict(X_test) 
dtest_predprob = model0.predict_proba(X_test)[:, 1]

#Print model report:
print "\nModel Report"
print "Accuracy : %.4g" % metrics.accuracy_score(dftest['Stable'].values, dtest_predictions)
print "AUC Score (Test): %f" % metrics.average_precision_score(dftest['Stable'], dtest_predprob)
print 'ROC_AUC_Score: ', metrics.roc_auc_score(dftest['Stable'], dtest_predprob)


Model Report
Accuracy : 0.89
AUC Score (Test): 0.901043
ROC_AUC_Score:  0.949938574939


Now that GBC has been tuned to mimic XGBoost we can formulate PDP. 